In [5]:
# ==================================================
#  Dual-Axis (Layered) Map using Custom Lat/Lon Fields
#  Dataset: ncr_ride_bookings.csv
#  Author: Nitish (based on Tableau Task 8)
# ==================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go

# ------------------------------
# STEP 1: Load and Clean Data
# ------------------------------
file_path = "/content/ncr_ride_bookings.csv"
df = pd.read_csv(file_path)

# Standardize column names
df.columns = df.columns.str.strip().str.lower()

# Check if latitude and longitude fields exist
if not {'latitude', 'longitude'}.issubset(df.columns):
    print("⚠️ Dataset missing lat/lon columns — simulating sample geospatial data for NCR region...")

    # Simulate geo data for demonstration
    cities = ["Delhi", "Gurugram", "Noida", "Faridabad", "Ghaziabad"]
    city_coords = {
        "Delhi": [28.6139, 77.2090],
        "Gurugram": [28.4595, 77.0266],
        "Noida": [28.5355, 77.3910],
        "Faridabad": [28.4089, 77.3178],
        "Ghaziabad": [28.6692, 77.4538]
    }

    np.random.seed(42)
    df["city"] = np.random.choice(cities, len(df))
    df["latitude"] = df["city"].apply(lambda c: city_coords[c][0] + np.random.uniform(-0.015, 0.015))
    df["longitude"] = df["city"].apply(lambda c: city_coords[c][1] + np.random.uniform(-0.015, 0.015))
    df["sales"] = np.random.randint(100, 2000, len(df))
else:
    # Assume there's a numeric sales field
    if "sales" not in df.columns:
        df["sales"] = np.random.randint(100, 1000, len(df))

# ------------------------------
# STEP 2: Aggregate Sales by City
# ------------------------------
if "city" in df.columns:
    city_sales = df.groupby(["city", "latitude", "longitude"], as_index=False)["sales"].sum()
else:
    # If no city field, aggregate by lat/lon
    city_sales = df.groupby(["latitude", "longitude"], as_index=False)["sales"].sum()
    city_sales["city"] = ["Location " + str(i) for i in range(len(city_sales))]

# ------------------------------
# STEP 3: Create Layer 1 (Sales by City)
# ------------------------------
layer1 = go.Scattergeo(
    lon=city_sales["longitude"],
    lat=city_sales["latitude"],
    text=city_sales["city"] + "<br>Sales: " + city_sales["sales"].astype(str),
    marker=dict(
        size=(city_sales["sales"] / city_sales["sales"].max()) * 50,
        color=city_sales["sales"],
        colorscale="Blues",
        cmin=city_sales["sales"].min(),
        cmax=city_sales["sales"].max(),
        colorbar_title="Sales (USD)",
        showscale=True,
        opacity=0.8,
        line=dict(width=0.5, color="black")
    ),
    name="Sales by City"
)

# ------------------------------
# STEP 4: Create Layer 2 (Individual Bookings)
# ------------------------------
layer2 = go.Scattergeo(
    lon=df["longitude"],
    lat=df["latitude"],
    mode="markers",
    text=df.get("city", "Booking"),
    marker=dict(size=4, color="orange", opacity=0.6),
    name="Individual Bookings"
)

# ------------------------------
# STEP 5: Combine (Dual Axis Map)
# ------------------------------
fig = go.Figure()

# Add both map layers
fig.add_trace(layer1)
fig.add_trace(layer2)

# Configure map projection and layout
fig.update_layout(
    title="🗺️ Dual-Axis (Layered) Map — Sales & Individual Bookings (Python Equivalent of Tableau)",
    geo=dict(
        projection_type="mercator",
        center=dict(lat=28.6, lon=77.2),
        scope="asia",
        lonaxis_range=[76.8, 77.6],
        lataxis_range=[28.3, 28.9],
        showland=True,
        landcolor="rgb(240, 240, 240)",
        countrycolor="gray",
    ),
    legend=dict(x=0.02, y=0.98),
    height=600,
    margin={"r":0, "t":40, "l":0, "b":0}
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df.describe()

,avg vtat,avg ctat,cancelled rides by customer,cancelled rides by driver,incomplete rides,booking value,ride distance,driver ratings,customer rating,latitude,longitude,sales
count,11439.000000,8397.000000,856.0,2186.0,745.0,8397.000000,8397.000000,7652.000000,7652.000000,12324.000000,12324.000000,12324.000000
mean,8.452601,29.108443,1.0,1.0,1.0,510.514350,24.889539,4.225026,4.405332,28.538173,77.279615,1043.731743
std,3.785855,8.934568,0.0,0.0,0.0,402.259066,14.086762,0.440241,0.438373,0.096562,0.151254,550.170658
min,2.000000,10.000000,1.0,1.0,1.0,50.000000,1.040000,3.000000,3.000000,28.393900,77.011614,100.000000
25%,5.300000,21.400000,1.0,1.0,1.0,237.000000,12.670000,4.000000,4.200000,28.452083,77.200986,563.000000
50%,8.200000,28.700000,1.0,1.0,1.0,414.000000,24.110000,4.300000,4.500000,28.535830,77.317757,1040.000000
75%,11.200000,36.900000,1.0,1.0,1.0,690.000000,37.160000,4.500000,4.800000,28.622290,77.399575,1523.000000
max,20.000000,45.000000,1.0,1.0,1.0,4277.000000,50.000000,5.000000,5.000000,28.684181,77.468799,1999.000000
